In [2]:
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
##############################################
##### PARTIE 1 : IMPORTATION DES DONNEES #####
##############################################

In [3]:
#### On importe le fichier répertoriant les installations industrielles rejetant des polluants

import requests

# Téléchargement du fichier avec la localisation des installations industrielles rejetant des polluants, directement depuis le site Géorisques
url_industries_polluantes = "https://files.georisques.fr/irep/2023.zip"
response_industries_polluantes = requests.get(url_industries_polluantes)

# Enregistrement du fichier localement
with open("Industries_polluantes.xlsx", "wb") as f:
    f.write(response_industries_polluantes.content)

# On importe le fichier

import zipfile

with zipfile.ZipFile("Industries_polluantes.xlsx", "r") as z:
    with z.open("2023/etablissements.csv") as f:
        industries_polluantes = pd.read_csv(f, sep=None, engine='python')

# On renomme la colonne insee dès maintenant (pour harmoniser ensuite)
industries_polluantes = industries_polluantes.rename(columns={"code_insee" : "insee"})
industries_polluantes.head()

# On importe le fichier des rejets
with zipfile.ZipFile("Industries_polluantes.xlsx", "r") as z:
    with z.open("2023/rejets.csv") as f:
        rejets = pd.read_csv(f, sep=None, engine='python')
rejets = rejets.rename(columns = {"code_insee" : "insee"})
rejets.head()

# On merge les deux tables
etablissements_rejets = pd.merge(industries_polluantes, rejets, on=["identifiant", "nom_etablissement", "code_postal", "insee",
"commune", "code_departement", "departement", "code_region", "region"], how="inner")
etablissements_rejets.head()

,identifiant,nom_etablissement,numero_siret,adresse,code_postal,insee,commune,code_departement,departement,code_region,...,coordonnees_x,coordonnees_y,code_epsg,code_ape,libelle_ape,code_eprtr,libelle_eprtr,annee_rejet,rejet_raccorde_m3_par_an,rejet_isole_m3_par_an
0,6101980,FERROGLOBE FRANCE,64200517700265,USINE D'ANGLEFORT,1350,01010,ANGLEFORT,01,AIN,84.0,...,5.81144,45.90610,4326.0,2410Z,Sidérurgie,2.(e).(i),destinées à la production de métaux bruts non ...,2023,NaN,2253116.0
1,6101989,KEM ONE,53869504000039,258 route de Saint Maurice de Gourdans,1360,01027,BALAN,01,AIN,84.0,...,5.10000,45.84721,4326.0,2014Z,Fabrication d'autres produits chimiques organi...,4.(a).(viii),"matières plastiques de base (polymères, fibres...",2023,NaN,685463.0
2,6112329,SK FUNCTIONAL POLYMER,87995793400059,258 route de Saint Maurice de Gourdans,1360,01027,BALAN,01,AIN,84.0,...,5.10000,45.84721,4326.0,2016Z,Fabrication de matières plastiques de base,4.(a).(viii),"matières plastiques de base (polymères, fibres...",2023,NaN,94045.0
3,6108136,ELIAN,76020009700060,Z.I. Sud Plastics Vallée - BP 59,1100,01031,BELLIGNAT,01,AIN,84.0,...,5.62876,46.24919,4326.0,2012Z,Fabrication de colorants et de pigments,NaN,NaN,2023,1866.0,NaN
4,6102002,SYND INTERCOMMUNAL DE VALORISATION,25740162000030,"Z.I. d'ARLOD, 5 chemin de Tapey",1200,01033,BELLEGARDE-SUR-VALSERINE,01,AIN,84.0,...,5.81411,46.08494,4326.0,3821Z,Traitement et élimination des déchets non dang...,5.(b),Installations destinées à l'incinération des d...,2023,NaN,25903484.0


In [5]:
#### On importe le fichier répertoriant les catastrophes naturelles de type inondations reconnues pour chaque commune

# Téléchargement du fichier avec le nombre de catastrophes reconnues pour chaque commune, directement depuis le site Géorisques
url_inondations = "https://files.georisques.fr/onrn/2025/ONRN_Reco_INON_8224.xlsx"
response_inondations = requests.get(url_inondations)

# Enregistrement du fichier localement
with open("ONRN_Reco_INON_8224.xlsx", "wb") as f:
    f.write(response_inondations.content)

# Importation et ouverture avec Pandas
inondations = pd.read_excel("ONRN_Reco_INON_8224.xlsx", sheet_name="Nb reco. Cat Nat")
inondations.head()

,Code INSEE,Commune,"Nombre de reconnaissances Cat Nat\n inondation (tous types)\n(Somme : coulée de boue, remontée de nappe, submersion marine)"
0,01001,L'Abergement-Clémenciat,2
1,01002,L'Abergement-de-Varey,2
2,01004,Ambérieu-en-Bugey,4
3,01005,Ambérieux-en-Dombes,2
4,01006,Ambléon,1


In [ ]:
### Préparation du dataframe de travail

# On renomme la colonne avec le code INSEE dans le df des communes (shapefiles) pour le joindre à celui tiré de Géorisques
communes = communes.rename(columns={"insee" : "Code INSEE"})
communes.head()

# On fait la jointure
df = pd.merge(communes, inondations, on="Code INSEE")

# On supprime la colonne wikipedia (inutile) et nom (identique à la colonne 'Commune')
del df['wikipedia']
del df['nom']
df.head()

# On réorganise le df (ordre des colonnes)
df = df.iloc[:,[3,0,1,2,4]]

# On renomme la dernière colonne (pour pouvoir l'appeler facilement)
df.columns = ['Commune', 'Code INSEE', 'surf_ha', 'geometry', 'Occurrences'] 

# On remplace "Pas de reconnaissance" par 0 (on transforme la ligne en numérique : les erreurs (ie les caractères) sont alors convertis en 0)
df["Occurrences"] = pd.to_numeric(df["Occurrences"], errors="coerce").fillna(0)

In [ ]:
#### On importe le fichier avec les shapefiles des communes

# Téléchargement du fichier avec les shapefiles des communes, directement depuis Datagouv
url_communes = "https://www.data.gouv.fr/api/1/datasets/r/0e117c06-248f-45e5-8945-0e79d9136165"
zip_communes = "shapefiles.zip"

# Enregistrement du fichier localement
response_communes = requests.get(url_communes)
with open(zip_communes, "wb") as f:
    f.write(response_communes.content)

# Importation avec GeoPandas
communes = gpd.read_file(f"zip://{zip_communes}!communes-20220101.shp")
communes.head()

In [ ]:
## On veut fusionner le df communes_statut avec des informations tirées de etablissements_rejets
## On a un problème : les arrondissements ne sont pas décrits dans le fichier des communes (mais ils le sont dans etablissements_rejets)

# On fusionne pour Paris, Lyon et Marseille
etablissements_rejets["insee"] = etablissements_rejets["insee"].replace(["13203", "13205", "13208", "13209", "13210", "13211", "13214", "13215", "13216"],
"13055").astype(str)
etablissements_rejets["insee"] = etablissements_rejets["insee"].replace(["69383", "69384", "69387", "69388", "69389"],"69123").astype(str)
etablissements_rejets["insee"] = etablissements_rejets["insee"].replace(["75101", "75109", "75111", "75112", "75115"],"75056").astype(str)

# On supprime les autres
etablissements_rejets.drop(etablissements_rejets[etablissements_rejets["insee"].isin(["14443", "27676", "33267", "49204", "49274", "49276", "50163",
"50460", "50602", "61356", "74093", "74181", "85212"])].index, inplace = True)

In [ ]:
### Préparation du dataframe de travail

# On fait la jointure
df = pd.merge(communes, etablissements_rejets, on="insee", how = "inner")

# Conversion du fichier au format GeoPandas
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Création d'un dataframe avec seulement les communes polluées
communes_polluees = gdf.copy()
communes_polluees = communes_polluees[communes_polluees["insee"].isin(etablissements_rejets["insee"])]

# On crée un df avec les communes et une variable polluée ou non (binaire)
communes_statut = communes.copy()
communes_statut["Polluee"] = communes_statut["insee"].isin(communes_polluees["insee"])

# On regarde combien on en a (2 373 polluées parmi 34 955 : environ 7% (c'est bien !?))
print(communes_statut["Polluee"].sum(), communes.shape)

In [ ]:
## On va réduire le df etablissements_rejets

# On regarde la taille et combien de communes uniques
print(etablissements_rejets.shape, etablissements_rejets["insee"].nunique(), etablissements_rejets.columns)

# On regroupe par la commune et on somme les rejets
etablissements_rejets_agr = etablissements_rejets.groupby("insee").agg(
    rejets_tot=("rejet_isole_m3_par_an", "sum"),
    nb_etablissements=("rejet_isole_m3_par_an", "count")
).reset_index()

In [ ]:
# On fusionne etablissements_rejets_agr et communes_statut
communes_statut = pd.merge(communes_statut, etablissements_rejets_agr, on = "insee")

# On ne conserve que les villes qui ont un rejet isolé positif
communes_statut = communes_statut[communes_statut["rejets_tot"] > 0]

In [ ]:
### On se concentre sur l'idf

# On crée un df communes seulement avec l'idf
communes["departement"] = communes["insee"].astype(str).str.slice(0,2)
communes_idf = communes.copy()
communes_idf = communes_idf[communes_idf["departement"].isin(["75", "77", "78", "91", "92", "93", "94", "95"])]

communes_polluees["departement"] = communes_polluees["insee"].astype(str).str.slice(0,2)
communes_polluees_idf = communes_polluees.copy()
communes_polluees_idf = communes_polluees_idf[communes_polluees_idf["departement"].isin(["75", "77", "78", "91", "92", "93", "94", "95"])]

# On crée un df avec les communes d'IDF et une variable polluée ou non (binaire)
communes_idf_statut = communes_idf.copy()
communes_idf_statut["Polluee"] = communes_idf["insee"].isin(communes_polluees_idf["insee"])

# On regarde combien on en a (157 polluées parmi 1 268 : environ 10% (c'est cool !))
print(communes_idf_statut["Polluee"].sum(), communes_idf.shape)

In [ ]:
#### On importe le dossier complet de l'INSEE pour apparier avec des données "sociales"

# On télécharge le dossier qui contient notre csv
url_complet = "https://www.insee.fr/fr/statistiques/fichier/5359146/dossier_complet.zip"
zip_path = "dossier_complet.zip" 

# On enregistre le fichier localement
response = requests.get(url_complet)
with open(zip_path, "wb") as f:
    f.write(response.content)

# On importe notre fichier avec Pandas (et on l'appelle complet_insee)
csv_complet_insee = "dossier_complet.csv" 
with zipfile.ZipFile(zip_path) as z:
    with z.open(csv_complet_insee) as f:
        complet_insee = pd.read_csv(f, sep=';', encoding='utf-8')

In [ ]:
#### On fait à nouveau la jointure avec notre df précédent

# On renomme la variable CODGEO en "insee" pour que ce soit homogène
complet_insee = complet_insee.rename(columns={"CODGEO":"insee"})

# On s'assure que la variable "insee" est similaire dans les deux tableaux
communes_idf_statut['insee'] = communes_idf_statut['insee'].astype(str).str.zfill(5)                     # zfill remplit de zéros à gauche pour atteindre 5 signes (pour harmoniser à la norme INSEE)
complet_insee['insee'] = complet_insee['insee'].astype(str).str.zfill(5)

# On fusionne
communes_idf_complet = pd.merge(communes_idf_statut, complet_insee, on="insee") # on obtient un tableau à 1982 colonnes (et 1 267 lignes)
communes_idf_complet.shape